# Transfer Learning usando lasagne - parte 2

Nesse tutorial, vamos utilizar CNNs treinadas na base ImageNet para outros problemas de classificação de imagens, usando transfer learning.

O objetivo é treinar um modelo para discriminar um subconjunto de 5 classes da base de dados Caltech 101 (http://www.vision.caltech.edu/Image_Datasets/Caltech101/). A base para esse exercício possui 325 imagens de tamanho 224x224x3. 

Na parte 2 desse tutorial, vamos utilizar o método de Finetunning descrito em [1]: 
 1. Vamos utilizar uma rede treinada na base ImageNet: https://github.com/Lasagne/Recipes/tree/master/modelzoo
 2. Vamos remover a última camada da rede, adicionar novas camadas e prosseguir com o treinamento do modelo


[1] Oquab, Maxime, et al. "Learning and transferring mid-level image representations using convolutional neural networks." [link](http://www.cv-foundation.org/openaccess/content_cvpr_2014/papers/Oquab_Learning_and_Transferring_2014_CVPR_paper.pdf)

In [ ]:
import cPickle
import lasagne
import theano.tensor as T
import theano
import numpy as np
from train import train_minibatch, plot_train_curves
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

## Dataset e pré-processamento

Para esse exercício, vamos utilizar a mesma base de dados (5 classes da Caltech-101). 

Primeiramente, vamos carregar a base de dados, e executar os passos de pré-processamento que utilizamos na parte 1

In [ ]:
data = np.load('caltech_5classes.npz')
x_train = data['x_train']
y_train = data['y_train']
x_test = data['x_test']
y_test = data['y_test']
classes = data['classes']

In [ ]:
from sklearn.cross_validation import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
params = cPickle.load(open('vgg_cnn_s.pkl'))
mean_img = params['mean image']

def process_dataset(x):
    x = np.transpose(x, (0,3,1,2))  # Modifica dados para: exemplos x canais RGB x altura x largura
    x = x[:, ::-1]                  # Modifica canais de RGB para BGR
    
    x = x - mean_img
    return x

x_train = process_dataset(x_train)
x_valid = process_dataset(x_valid)
x_test = process_dataset(x_test)

Vamos agora implementar o método de fine-tunning. 

![](images/oquab.png)

Para isso, precisamos dos seguintes passos:

## Criação do novo modelo

1. Criar o modelo de CNN que foi usado na base de dados origem (vgg_cnn_s)
2. Carregar os pesos do modelo
3. Alterar o modelo:
   * Remover a última camada
   * Adicionar uma ou mais camadas ao fim
   
## Treinamento
* Fazer o fine-tunning the todas as camadas **ou**
* Fazer o treinamento das últimas camadas que foram adicionadas


## Criação do novo modelo

Vamos começar importando o modelo já treinado, e verificando as camadas que ele possui:

In [ ]:
from vgg_cnn_s_cpu import build_model

In [ ]:
model = build_model()

In [ ]:
model.keys()

## Exercício - alteração do modelo

Vamos criar uma função que:
 * Carregue o modelo treinado na base origem (já está implementado abaixo)
 * Delete a última camada (fc8 e prob)
 * Crie uma nova camada (DenseLayer), com 5 saídas, e não-linearidade softmax. Chame-a de net['out']

In [ ]:
from lasagne.layers import DenseLayer
from lasagne.nonlinearities import softmax


In [ ]:
# Sua solução:


def build_model_for_finetuning(params):
    model = build_model()
    lasagne.layers.set_all_param_values(model['prob'], params['values'])
    
    ## Coloque aqui seu código para deletar as últimas camadas e criar nova(s) camada(s). 
    # Nomeie a última camada como "model['out']
    
    
    
    return model

In [ ]:
%load solutions/transfer_build_model.py

In [ ]:
model = build_model_for_finetuning(params)

assert 'out' in model, 'Ultima camada deveria se chamar "out"'
assert model['out'].input_layer == model['drop7'], 'Ultima camada deveria receber dados da camada "drop7"'


# Fine-tuning - treinamento usando regularização

Num primeiro momento, vamos considerar o caso de fazer o "fine-tuning" de todas as camadas. Isto é, a CNN é iniciada com os pesos aprendidos na base de origem, e agora faremos o treinamento na base destino, normalmente usando uma Learning Rate menor.

Vamos usar a função de custo que utilizamos ontem (cross-entropy) com uma variação: vamos adicionar regularização.

Regularização é importante para evitar over-fitting, e em particular é útil quando a base de dados é pequena (que é o nosso caso agora). A forma de regularização mais comum é conhecida como "Norma L2", que adiciona uma penalidade para valores grandes de $\textbf{w}$:

$$L_\text{reg} = L + \sum_i w_i^2$$

Em lasagne, podemos implementá-lo da seguinte forma:

```
loss = ... #função para calcular a funcao de custo, como anteriormente
loss += w_decay * regularize_layer_params(ultima_camada, l2)
```


In [ ]:
from lasagne.regularization import l2, regularize_layer_params

## Exercício - implementando regularização L2

Nesse exercício, vamos atualizar a função que compila a função de treinamento, para incluir regularização L2. Para isso, utilize a função ```regularize_layer_params``` para computar $\sum_i w_i^2$, e some esse valor à função de custo.



In [ ]:
def compile_train_function(net, lr, w_decay):
    input_var = net['input'].input_var
    output_var = T.ivector()

    probs = lasagne.layers.get_output(net['out'], inputs=input_var)
    loss = lasagne.objectives.categorical_crossentropy(probs, output_var)
    loss = loss.mean()
    
    ####
    # insira aqui o código para adicionar regularização à função de custo
    
    
    ###
    
    y_pred = T.argmax(probs, axis=1)
    acc = T.eq(y_pred, output_var)
    acc = acc.mean()
    
    test_probs = lasagne.layers.get_output(net['out'], inputs=input_var, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_probs, output_var)
    test_loss = test_loss.mean()
    
    test_pred = T.argmax(test_probs, axis=1)
    test_acc = T.eq(test_pred, output_var)
    test_acc = test_acc.mean()

    params = lasagne.layers.get_all_params(net['out'])
    updates = lasagne.updates.sgd(loss, params, lr)

    train_fn = theano.function([input_var, output_var], [loss, acc], updates=updates)
    val_fn = theano.function([input_var, output_var], [test_loss, test_acc])
    return train_fn, val_fn

Note que na função acima, usamos variáveis diferentes para a função de custo em treinamento e validação. Isso é importante para modelos que usem camadas não-determinísticas (e.g. Dropout), que é o caso do modelo atual. Essas camadas possuem comportamento diferente para treinamento e teste, portanto é importante obter as saídas de treinamento e validação da seguinte forma:

```
train_probs = lasagne.layers.get_output(net['out'], inputs=input_var)
test_probs = lasagne.layers.get_output(net['out'], inputs=input_var, deterministic=True)
```



In [ ]:
#Execute para ver a solução

%load solutions/transfer_train.py

## Exercício: finetuning

* Crie um modelo chamando a função "build_model_for_finetuning".
* Compile as funções de treinamento usando a função acima, com lr=0.001 e w_decay= 1e-5
* Execute a célula seguinte para efetuar o treinamento (levará vários minutos para treinar)

In [ ]:
# Sua solução

In [ ]:
%load solutions/transfer_model.py

In [ ]:
train_curves = train_minibatch(train_fn, valid_fn,    
                     train_set=(x_train, y_train), 
                     valid_set=(x_test, y_test),
                     epochs=20,
                     batch_size=16)
plot_train_curves(train_curves)

A performance em aceitação é razoável, mas notamos que o modelo entra em overfitting - principalmente causado pelo pequeno tamanho da base de treinamento

## Re-treinando apenas algumas camadas

Vamos agora considerar o caso de re-treinar apenas um sub-conjunto de camadas (geralmente, as últimas camadas adicionadas ao modelo)

### Exercício: treinando apenas algumas camadas

Modifique a função de treinamento abaixo, para que o treinamento atualize apenas os pesos de algumas camadas da rede.

Isto é, ao invés de utilizar a função ```lasagne.layers.get_all_params``` para obter a lista de parametros, vamos contruir a lista de parâmetros manualmente. Dada uma lista de camadas que desejamos treinar, agregue todos os parâmetros dessas camadas em uma lista chamada "params"

Dica: utilize a função abaixo [manual](http://lasagne.readthedocs.io/en/latest/modules/layers/base.html#lasagne.layers.Layer.get_params)

```
camada.get_params(treinable=True)
``` 

In [ ]:
def compile_train_function_somelayers(net, lr, w_decay, layers_to_train):
    input_var = net['input'].input_var
    output_var = T.ivector()

    probs = lasagne.layers.get_output(net['out'], inputs=input_var)
    loss = lasagne.objectives.categorical_crossentropy(probs, output_var)
    loss = loss.mean()
    loss += w_decay * regularize_layer_params(net['out'], l2)
    
    y_pred = T.argmax(probs, axis=1)
    acc = T.eq(y_pred, output_var)
    acc = acc.mean()
    
    test_probs = lasagne.layers.get_output(net['out'], inputs=input_var, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_probs, output_var)
    test_loss = test_loss.mean()
    
    test_pred = T.argmax(test_probs, axis=1)
    test_acc = T.eq(test_pred, output_var)
    test_acc = test_acc.mean()
    
    ####
    # Adicione aqui o codigo para criar uma lista chamada "params" com os 
    # parametros de todas as camadas da lista "layers_to_train"
    
    ####
        
    updates = lasagne.updates.sgd(loss, params, lr)

    train_fn = theano.function([input_var, output_var], [loss, acc], updates=updates)
    val_fn = theano.function([input_var, output_var], [test_loss, test_acc])
    return train_fn, val_fn

Vamos inicialmente treinar apenas a última camada:

In [ ]:
model = build_model_for_finetuning(params)

train_fn, valid_fn = compile_train_function_somelayers(model, lr=0.005, w_decay=1e-5, layers_to_train=[model['out']])

In [ ]:
#Forçar a liberação de memória do Garbage Collector - para evitar problemas de memória

import gc
gc.collect()

In [ ]:
train_curves = train_minibatch(train_fn, valid_fn,     # Treinamento usando Batch Gradient Descent
                     train_set=(x_train, y_train), 
                     valid_set=(x_test, y_test),
                     epochs=20,
                     batch_size=16)


In [ ]:
plot_train_curves(train_curves)

## Exercício: treinando as duas ultimas camadas

* Crie um modelo usando ```build_model_for_finetuning(params)```
* Treine as duas últimas camadas, usando lr=0.005, w_decay=1e-5, por 20 epochs e batch_size=16
* Mostre o gráfico da curva de aprendizagem, usando a função ```plot_train_curves```

In [ ]:
#Sua solução

In [ ]:
%load solutions/transfer_train_twolayers.py

Lembre-se que nesse exercício usamos uma base de destino pequena. O que você observa ao treinar todas as camadas vs treinar somente a nova camada nesse cenário?

# Exercícios extras:

* A.1 Adicionar mais camadas ao fim da rede - verificar a performance treinando apenas essas camadas, ou todas as camadas da rede